
# Problem Statement:
we have extracted tweets from customers about various tech firms who manufacture and sell mobiles, computers , laptops, etc the task is to identify if the tweets have a negative sentiment towards such companies or products

## About Dataset
we have trian and text dataset. Train data set has both input and output variable.
we need to classify the tweeta in test dataset.

Number of observations in Train Set : 7,920
Number of observations in Test Set : 1,953

## Data Download

You can download the dataset from our[DataHack Problem](https://datahack.analyticsvidhya.com/contest/practice-problem-identify-the-sentiments/)

## My Approch to the problem


1.   Pre-Process the data
2.   Define the architecture of you model
3.   Since it is a binary classification problem. make sure that you use the 'sigmoid' activation funxtion in the output layer
4.   Train the model
5.   Do the same preprocessing on the test dataset.
6.   generate prediction for the test set using trained model
7.   save the predictions in the csv file.


# Dataset contain 
0 - Positive
1 - Negative

# 1. Import Libraries and Load Dataset

In [117]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score

In [118]:
train = pd.read_csv("/content/drive/MyDrive/RNN ASSIGNMENT DATASET/train_2kmZucJ.csv")
test = pd.read_csv("/content/drive/MyDrive/RNN ASSIGNMENT DATASET/test_12QyDcx.csv")
sample = pd.read_csv("/content/drive/MyDrive/RNN ASSIGNMENT DATASET/sample_submission_LnhVWA4.csv")

In [119]:
train.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...


In [120]:
train.iloc[0,2]

'#fingerprint #Pregnancy Test https://goo.gl/h1MfQV #android #apps #beautiful #cute #health #igers #iphoneonly #iphonesia #iphone'

In [121]:
train['tweet'].sample(10)

3747    Smile!!! #asian #filipina #pinay #picstich #fa...
3360    #Apple says you can't give away #iPhone or #iP...
1329    • new baby • #amazing #best #beautiful #iPhone...
5122    New #Coffee Instagram by @estradafx #instaself...
791     Shot on iPhone X . . #iphone #apple #iphonex #...
943     After my trial of #Apple #iPhone I will not be...
1289    Why is there always a new version of iTunes to...
3614    I'm inlove with this Family. #Apple-Family. ne...
5079    New wallpapers #wall #wallpaper #home #galaxy ...
6004    Ils sont content . #LizardSquad #PSN #XBOXLIVE...
Name: tweet, dtype: object

# 2. Preprocess the data

In [122]:
def cleaner(text):
    text = re.sub(r'http?s://[a-zA-Z0-9_.]+','',text)
    text = re.sub(r'@[a-zA-Z]+',' ',text)
    text = re.sub(r'[^a-zA-Z]',' ',text)
    text = text.lower()
    token = text.split()
    return " ".join(token)

In [123]:
train['clean_tweet'] = train['tweet'].apply(cleaner)

In [124]:
train.head()

,id,label,tweet,clean_tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...,fingerprint pregnancy test h mfqv android apps...
1,2,0,Finally a transparant silicon case ^^ Thanks t...,finally a transparant silicon case thanks to m...
2,3,0,We love this! Would you go? #talk #makememorie...,we love this would you go talk makememories un...
3,4,0,I'm wired I know I'm George I was made that wa...,i m wired i know i m george i was made that wa...
4,5,1,What amazing service! Apple won't even talk to...,what amazing service apple won t even talk to ...


In [125]:
train.shape

(7920, 4)

In [126]:
# check the test set
test.head()

,id,tweet
0,7921,I hate the new #iphone upgrade. Won't let me d...
1,7922,currently shitting my fucking pants. #apple #i...
2,7923,"I'd like to puts some CD-ROMS on my iPad, is t..."
3,7924,My ipod is officially dead. I lost all my pict...
4,7925,Been fighting iTunes all night! I only want th...


In [127]:
# sample
sample.head()

,id,label
0,7921,0
1,7922,0
2,7923,0
3,7924,0
4,7925,0


In [128]:
train.head()

,id,label,tweet,clean_tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...,fingerprint pregnancy test h mfqv android apps...
1,2,0,Finally a transparant silicon case ^^ Thanks t...,finally a transparant silicon case thanks to m...
2,3,0,We love this! Would you go? #talk #makememorie...,we love this would you go talk makememories un...
3,4,0,I'm wired I know I'm George I was made that wa...,i m wired i know i m george i was made that wa...
4,5,1,What amazing service! Apple won't even talk to...,what amazing service apple won t even talk to ...


In [129]:
x = []
for i in train['clean_tweet']:
    x.append(i)

In [130]:
x

['fingerprint pregnancy test h mfqv android apps beautiful cute health igers iphoneonly iphonesia iphone',
 'finally a transparant silicon case thanks to my uncle yay sony xperia s sonyexperias http instagram com p yget jc jm',
 'we love this would you go talk makememories unplug relax iphone smartphone wifi connect http fb me n lsupcu',
 'i m wired i know i m george i was made that way iphone cute daventry home http instagr am p li ujs k',
 'what amazing service apple won t even talk to me about a question i have unless i pay them for their stupid support',
 'iphone software update fucked up my phone big time stupid iphones',
 'happy for us instapic instadaily us sony xperia xperiaz p z qgfwlvj',
 'new type c charger cable uk http www ebay co uk itm bay amazon etsy new year rob cross toby young evemun mcmafia taylor spectre newyear starting recipes technology samsunggalaxys iphonex pic twitter com pjiwq wtc',
 'bout to go shopping again listening to music iphone justme music likeforli

In [131]:
token_count = []
for i in x:
    token_count.append(len(i.split()))

In [132]:
# checking the len of average sentences has
import statistics
import math
max_len = math.ceil(statistics.mean(token_count))+5
print(max_len)

25


In [133]:
y = train['label'].values

# 3.Data Preperation for model

In [134]:
x_tr , x_val ,y_tr , y_val = train_test_split(x,y, test_size = 0.2 , shuffle = True , random_state = 42)

In [135]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

x_tokenizer = Tokenizer()
x_tokenizer.fit_on_texts(x_tr)

In [136]:
count = len(x_tokenizer.word_index)

In [137]:
x_tokenizer = Tokenizer(num_words=count , oov_token='unk')
x_tokenizer.fit_on_texts(x_tr)

In [138]:
x_tr_seq = x_tokenizer.texts_to_sequences(x_tr)
x_val_seq = x_tokenizer.texts_to_sequences(x_val)

In [139]:
x_tr_seq  = pad_sequences(x_tr_seq, padding ='post', maxlen=max_len)
x_val_seq  = pad_sequences(x_val_seq, padding ='post', maxlen=max_len)

In [140]:
vocab_size = x_tokenizer.num_words+1
vocab_size

18760

In [141]:
x_tr_seq[0]

array([5898, 5899, 5900, 3854, 3855,  876, 2930,   13,    2, 3856,  465,
       3857,  624, 1252, 5901,  957, 2021, 2022, 3858,  228, 5902,   16,
          7, 5903, 5904], dtype=int32)

# 4. Model Architecture

In [81]:
from keras.models import Sequential
from keras.layers import *
from keras.callbacks import * 

In [82]:
model = Sequential()

model.add(Embedding(input_dim=vocab_size, output_dim=200,input_shape = (max_len,),trainable = True,mask_zero=True))

model.add(GRU(units=256))

model.add(Dense(units = 128, activation='relu'))

model.add(Dense(units = 1, activation='sigmoid'))


model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 25, 200)           3752000   
_________________________________________________________________
gru_1 (GRU)                  (None, 256)               351744    
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 4,136,769
Trainable params: 4,136,769
Non-trainable params: 0
_________________________________________________________________


In [83]:
# Compile the model
model.compile(optimizer = "adam" , loss = 'binary_crossentropy' , metrics = ['accuracy'])

In [84]:
# Define the model call backs to save the best features
model_callback = ModelCheckpoint("/content/drive/MyDrive/RNN ASSIGNMENT DATASET/SentimentAnalysisIIWeights.best.hdf5",
                                 save_best_only=True,
                                 monitor = "val_loss",
                                 mode ='min',
                                 verbose = 1)

In [85]:
# Train the model
model.fit(x_tr_seq, y_tr,batch_size = 64, epochs=10,validation_data=(x_val_seq,y_val),callbacks=[model_callback])

Epoch 1/10
99/99 [==============================] - 36s 137ms/step - loss: 0.4327 - accuracy: 0.8001 - val_loss: 0.2441 - val_accuracy: 0.8895

Epoch 00001: val_loss improved from inf to 0.24410, saving model to /content/drive/MyDrive/RNN ASSIGNMENT DATASET/SentimentAnalysisIIWeights.best.hdf5
Epoch 2/10
99/99 [==============================] - 12s 118ms/step - loss: 0.1562 - accuracy: 0.9386 - val_loss: 0.3056 - val_accuracy: 0.8687

Epoch 00002: val_loss did not improve from 0.24410
Epoch 3/10
99/99 [==============================] - 12s 117ms/step - loss: 0.0717 - accuracy: 0.9735 - val_loss: 0.8893 - val_accuracy: 0.7626

Epoch 00003: val_loss did not improve from 0.24410
Epoch 4/10
99/99 [==============================] - 12s 122ms/step - loss: 0.0283 - accuracy: 0.9902 - val_loss: 0.8140 - val_accuracy: 0.7216

Epoch 00004: val_loss did not improve from 0.24410
Epoch 5/10
99/99 [==============================] - 12s 119ms/step - loss: 0.0220 - accuracy: 0.9946 - val_loss: 1.8089 

In [86]:
# Load the model
model.load_weights("/content/drive/MyDrive/RNN ASSIGNMENT DATASET/SentimentAnalysisIIWeights.best.hdf5")
pred_prob = model.predict(x_val_seq)

In [87]:
pred_prob[0]

array([0.00058809], dtype=float32)

# finding the optmial value that will able to classify the model

In [142]:
def classify(pred_prob,thres):
    y_pred = []
    for i in pred_prob:
        if i > thres:
            y_pred.append(1)
        else:
            y_pred.append(0)
    return y_pred

In [143]:
y_pred = np.array(classify(pred_prob,0.5))

In [144]:
type(y_pred)

numpy.ndarray

In [145]:
type(y_val)

numpy.ndarray

In [146]:
from sklearn.metrics import f1_score, classification_report

print(f1_score(y_val,y_pred))

0.78300803673938


In [147]:
print(classification_report(y_val , y_pred))

              precision    recall  f1-score   support

           0       0.92      0.91      0.92      1152
           1       0.78      0.79      0.78       432

    accuracy                           0.88      1584
   macro avg       0.85      0.85      0.85      1584
weighted avg       0.88      0.88      0.88      1584



# show inference

In [148]:
def predict_tag(comment):

    text = []

    # preprocess text
    text = [cleaner(comment)]

    # convert to integers sequences
    seq = x_tokenizer.texts_to_sequences(text)

    # pad the sequences
    pad_seq = pad_sequences(seq, padding= 'post', maxlen=max_len)

    # make predictions
    pred_prob = model.predict(pad_seq)
    classes = np.array(classify(pred_prob, 0.5))

    # classes = np.array([classes])
    # classes = mlb.inverse_transform(classes)
    return classes

In [149]:
test.head()

,id,tweet
0,7921,I hate the new #iphone upgrade. Won't let me d...
1,7922,currently shitting my fucking pants. #apple #i...
2,7923,"I'd like to puts some CD-ROMS on my iPad, is t..."
3,7924,My ipod is officially dead. I lost all my pict...
4,7925,Been fighting iTunes all night! I only want th...


In [150]:
test.iloc[0,1]

"I hate the new #iphone upgrade. Won't let me download apps. #ugh #apple sucks"

In [151]:
r = predict_tag("I love iphone")
print(r)

[0]


In [152]:
test_gru = test
y_pred = []
for i in test_gru['tweet']:
    if predict_tag(i)==1:
        y_pred.append(1)
    else:
        y_pred.append(0)

In [101]:
test_gru['label'] = y_pred

In [115]:
test_gru.head()

,id,label
0,7921,1
1,7922,1
2,7923,1
3,7924,1
4,7925,1


In [103]:
test_gru.drop(['tweet'], axis =1,inplace=True)

In [104]:
test_gru.head()

,id,label
0,7921,1
1,7922,1
2,7923,1
3,7924,1
4,7925,1


In [105]:
test_gru.to_csv('resulted_GRU.csv', index = False)

# CNN MODEL FOR SENTIMENT ANALYSIS

In [106]:
model_conv = Sequential()
model_conv.add(Embedding(input_dim=vocab_size, output_dim=100,trainable = True,input_shape = (max_len,)))
model_conv.add(Conv1D(filters=64, kernel_size=3, padding='same'))
model_conv.add(Dropout(0.1))
model_conv.add(GlobalMaxPooling1D())
model_conv.add(Dense(units=128,activation='relu'))
model_conv.add(Dense(units=1,activation='sigmoid'))
model_conv.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 25, 100)           1876000   
_________________________________________________________________
conv1d (Conv1D)              (None, 25, 64)            19264     
_________________________________________________________________
dropout (Dropout)            (None, 25, 64)            0         
_________________________________________________________________
global_max_pooling1d (Global (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 129       
Total params: 1,903,713
Trainable params: 1,903,713
Non-trainable params: 0
____________________________________________

In [107]:
# Compile the model
model_conv.compile(optimizer = "adam" , loss = 'binary_crossentropy' , metrics = ['accuracy'])

# Define the model call backs to save the best features
model_conv_callback = ModelCheckpoint("/content/drive/MyDrive/RNN ASSIGNMENT DATASET/SentimentAnalysisIIWeights_CNN.best.hdf5",
                                 save_best_only=True,
                                 monitor = "val_loss",
                                 mode ='min',
                                 verbose = 1)
# Train the model
model_conv.fit(x_tr_seq, y_tr,batch_size = 64, epochs=10,validation_data=(x_val_seq,y_val),callbacks=[model_conv_callback])


# Load the model
model_conv.load_weights("/content/drive/MyDrive/RNN ASSIGNMENT DATASET/SentimentAnalysisIIWeights_CNN.best.hdf5")
pred_prob = model_conv.predict(x_val_seq)

Epoch 1/10
99/99 [==============================] - 5s 27ms/step - loss: 0.5246 - accuracy: 0.7599 - val_loss: 0.2881 - val_accuracy: 0.8794

Epoch 00001: val_loss improved from inf to 0.28811, saving model to /content/drive/MyDrive/RNN ASSIGNMENT DATASET/SentimentAnalysisIIWeights_CNN.best.hdf5
Epoch 2/10
99/99 [==============================] - 2s 24ms/step - loss: 0.2288 - accuracy: 0.9103 - val_loss: 0.2570 - val_accuracy: 0.8807

Epoch 00002: val_loss improved from 0.28811 to 0.25696, saving model to /content/drive/MyDrive/RNN ASSIGNMENT DATASET/SentimentAnalysisIIWeights_CNN.best.hdf5
Epoch 3/10
99/99 [==============================] - 2s 22ms/step - loss: 0.1138 - accuracy: 0.9642 - val_loss: 0.3201 - val_accuracy: 0.8611

Epoch 00003: val_loss did not improve from 0.25696
Epoch 4/10
99/99 [==============================] - 2s 23ms/step - loss: 0.0471 - accuracy: 0.9883 - val_loss: 0.4016 - val_accuracy: 0.8497

Epoch 00004: val_loss did not improve from 0.25696
Epoch 5/10
99/99

In [108]:
def classify(pred_prob,thres):
    y_pred = []
    for i in pred_prob:
        if i > thres:
            y_pred.append(1)
        else:
            y_pred.append(0)
    return y_pred

In [109]:
from sklearn.metrics import f1_score, classification_report

y_pred = np.array(classify(pred_prob,0.5))
print(f1_score(y_val,y_pred))
print(classification_report(y_val , y_pred))

0.78300803673938
              precision    recall  f1-score   support

           0       0.92      0.91      0.92      1152
           1       0.78      0.79      0.78       432

    accuracy                           0.88      1584
   macro avg       0.85      0.85      0.85      1584
weighted avg       0.88      0.88      0.88      1584



# Show Inference

In [113]:
def predict_tag_CNN(comment):

    text = []

    # preprocess text
    text = [cleaner(comment)]

    # convert to integers sequences
    seq = x_tokenizer.texts_to_sequences(text)

    # pad the sequences
    pad_seq = pad_sequences(seq, padding= 'post', maxlen=max_len)

    # make predictions
    pred_prob = model_conv.predict(pad_seq)
    classes = np.array(classify(pred_prob, 0.5))
    return classes

In [114]:
r = predict_tag_CNN("I love iphone")
print(r)

[0]


In [153]:
test_cnn = test
y_pred = []
for i in test_cnn['tweet']:
    if predict_tag(i)==1:
        y_pred.append(1)
    else:
        y_pred.append(0)

test_cnn['label'] = y_pred

In [154]:
test_cnn['label'] = y_pred

In [155]:
test_cnn.drop(['tweet'], axis =1,inplace=True)

In [158]:
test_cnn.to_csv('/content/drive/MyDrive/RNN ASSIGNMENT DATASET/resulted_cnn.csv', index = False)